In [1]:
import pandas as pd
import numpy as np

# 数据集描述
蘑菇数据集，包括毒性，大小，表面，颜色等，所有数据均为字符串类型，分析毒性与其他属性的关系
# 读取数据集

In [3]:
dataset = pd.read_csv("./mushrooms.csv")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
class                       8124 non-null object
cap-shape                   8124 non-null object
cap-surface                 8124 non-null object
cap-color                   8124 non-null object
bruises                     8124 non-null object
odor                        8124 non-null object
gill-attachment             8124 non-null object
gill-spacing                8124 non-null object
gill-size                   8124 non-null object
gill-color                  8124 non-null object
stalk-shape                 8124 non-null object
stalk-root                  8124 non-null object
stalk-surface-above-ring    8124 non-null object
stalk-surface-below-ring    8124 non-null object
stalk-color-above-ring      8124 non-null object
stalk-color-below-ring      8124 non-null object
veil-type                   8124 non-null object
veil-color                  8124 non-null object
ring-number

可以发现，一共包括23个属性，没有缺失值
# 直观分析——鲜艳颜色的蘑菇都有毒？

In [53]:
poison = dataset[dataset["class"] == "p"]["cap-color"]
not_poison = dataset[dataset["class"] != "p"]["cap-color"]
# print(pd.value_counts(not_poison))
poison_color = pd.concat([pd.value_counts(poison),pd.value_counts(not_poison),pd.value_counts(dataset["cap-color"])],axis=1,keys=["poison","normal","all"])
poison_color = poison_color.fillna(value=0)
# print(poison_color)
poison_color = poison_color.groupby(poison_color.columns,axis=1).apply(lambda x:x / x.sum())
# group.agg(lambda x:x / x.sum())
# poison_color["poison"] = poison_color["poison"] / poison_color["poison"].sum()
# poison_color["all"] = poison_color["all"] / poison_color["all"].sum()
print(poison_color.sort_values(by="poison").loc[["p","b","y","e"]])

     poison    normal       all
p  0.022472  0.013308  0.017725
b  0.030644  0.011407  0.020679
y  0.171604  0.095057  0.131955
e  0.223698  0.148289  0.184638


可得还是有一定道理的，尤其是黄色和红色的蘑菇

# 数据预处理——文字向量化
该数据集中，最重要的预处理就是将文字转换为向量